In [1]:
import os
from pypdf import PdfReader
import csv
import pandas as pd
import numpy as np
import re

In [2]:
# d = {'Invoice_#':[0], 'Invoice_Total':[0]}
# df = pd.DataFrame(data=d)
# df

In [3]:
# Import Excel created from net32 orders
df = pd.read_excel('raw.xlsx')
df

,Net32_ID,Order,Order Date,Customer Name,Net32 Total
0,net32-45929949,45929949,2024-07-08,Dominic Armbrust,55.32
1,net32-45932743,45932743,2024-07-08,Jacob Hamby,40.66
2,net32-45934613,45934613,2024-07-08,Sandeep Jagini,136.39
3,net32-45940421,45940421,2024-07-08,michael woody,62.98
4,net32-45946669,45946669,2024-07-08,Will Catterton,138.21
5,net32-45959726,45959726,2024-07-09,Michelle Bargfrede,61.64
6,net32-45960969,45960969,2024-07-09,Monique Garcia,62.35
7,net32-45972871,45972871,2024-07-09,Natalie Erskine,61.93
8,net32-45983288,45983288,2024-07-10,charles smith,123.27
9,net32-46002824,46002824,2024-07-10,Patti Britton,98.26


In [4]:
directory_in_str = "PDF_invoices"
directory = os.fsencode(directory_in_str)

df.insert(loc = 0, column = 'Invoice', value = '')
df.insert(loc = 1, column = 'Invoice_total', value = '')
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print("------------\n\n",type(filename))
    if filename.endswith(".pdf"):
        print(filename)
        reader = PdfReader("PDF_invoices/"+filename)
        text = "" # empty string
        for page in reader.pages:
            text += page.extract_text() + "\n"
            print(text)
            # # Extract order #
            net32_order_obj = re.search("(?<=net32-).*?(?=\s)",text)
            net32_order_str = net32_order_obj.group()
            net32_order_int = int(net32_order_str)
            print("Order Object: ", net32_order_obj, "Order String: ", net32_order_str)            
            # # Extract invoice number and save to df
            net32_invoice_obj = re.search("(?<=INC_ - ).*?(?=\.)", filename)
            net32_invoice_str = net32_invoice_obj.group()
            print("Invoice Object: ", net32_invoice_obj, "Invoice String: ", net32_invoice_str)
            # # Extract total price
            net32_total_obj = re.search("(?<=USD ).*?(?=\s)", text)
            net32_total_str = net32_total_obj.group()
            print("Total Object: ", net32_total_obj, "Total String: ", net32_total_str)
            # # Save net32 invoice number and total cost to existing, raw df
            idx = df.loc[df['Order'] == net32_order_int].index
            print("Index: ", idx[0])            
            df.loc[idx[0], 'Invoice'] = net32_invoice_str
            df.loc[idx[0], 'Invoice_total'] = net32_total_str
            # # Save file name as net32 order #
            os.rename("PDF_invoices/"+filename, "PDF_invoices/"+net32_invoice_str + '.pdf')

------------

 <class 'str'>
A_R Invoice -MEDIDENTA INTERNATIONAL , INC_ - 14064785.pdf
Invoice
Bill To : Ship To :
P.O. Number Payment Terms Ship Date F.O.B.
Amount Unit Price Description Item # ShippedInvoice Date Due Date Invoice No.
Ship ViaPac-Dent, Inc.
670 Endeavor Circle
Brea, CA 92821, USA
(909) 839-0888
S.O. No. Account No.
Order Qty B/OPacking Slip #
07/09/2024 08/08/2024  14064785  13066265
MEDIDENTA INTERNATIONAL , INC. MEDIDENTA INTERNATIONAL , INC.
MEDIDENTA
3575 W POST RD
LAS VEGAS NV  89118
USADominic Armbrust
2737 N 182ND ST
ELKHORN, NE 68022-1599
C00151  12052333 net32-45929949 NET 30 US MAIL
UOM07/14/2024
BP Cat . #
 3  0  8.39  3  25.17 EZAim Disposable 
Adhesive Bite Tab Holders 
Universal Red - Compared 
to Schick AimRight 
B1073051 50/PackBag 131113
Tracking #:
943463610605144342566
0
Package Information:
NET32 ORDER
TOTAL :Subtotal
Sales Tax
Freight Fee
Total Invoiced Amount
Payment/ Credit Applied
USD 35.98 0.00
 35.98
 0.00 25.17
Dropship Fee
Packing Fee 0.00

In [5]:
df

,Invoice,Invoice_total,Net32_ID,Order,Order Date,Customer Name,Net32 Total
0,14064785,35.98,net32-45929949,45929949,2024-07-08,Dominic Armbrust,55.32
1,14064786,27.48,net32-45932743,45932743,2024-07-08,Jacob Hamby,40.66
2,14064787,73.62,net32-45934613,45934613,2024-07-08,Sandeep Jagini,136.39
3,14064788,32.34,net32-45940421,45940421,2024-07-08,michael woody,62.98
4,14064789,73.39,net32-45946669,45946669,2024-07-08,Will Catterton,138.21
5,14064790,33.55,net32-45959726,45959726,2024-07-09,Michelle Bargfrede,61.64
6,14064791,32.34,net32-45960969,45960969,2024-07-09,Monique Garcia,62.35
7,14064870,33.55,net32-45972871,45972871,2024-07-09,Natalie Erskine,61.93
8,14064873,55.95,net32-45983288,45983288,2024-07-10,charles smith,123.27
9,14064974,60.03,net32-46002824,46002824,2024-07-10,Patti Britton,98.26


## Note: The below code is simply for testing.

Example file name:
A_R Invoice -MEDIDENTA INTERNATIONAL , INC_ - 14064785.pdf

In [8]:
# Note: this extracts every char after the input until getting to the first empty space
try1 = re.search("(?<=potato ).*?(?=\s)", 'potato watko rewre')
print(try1)

<re.Match object; span=(7, 12), match='watko'>


In [9]:
# Note: this extracts every char after the input until getting to the first dot '.')
try2 = re.search("(?<=INC_ - ).*?(?=\.)",'A_R Invoice -MEDIDENTA INTERNATIONAL , INC_ - 14064785.pdf ')
print(type(try2),try2.group())

<class 're.Match'> 14064785


In [10]:
try0 = re.search("(?<=USD ).*?(?=\s)", 'Payment/ Credit Applied USD 35.98 0.00  35.98')
print(try0)

<re.Match object; span=(28, 33), match='35.98'>
